Indehold:

 - Assess your predicted strain designs using simulations and pathway visualizations
 - Assess manually derived strain designs using model simulations
 - Based on your computations, provide a top 10 list of most promising cell factory designs. The criteria for "most promising" are the number of modifications, yield, growth rate and others you might define

In [123]:
import cobra
from cobra.io import read_sbml_model 
from cobra import Model, Reaction, Metabolite
import escher
from escher import Builder
import json

In [124]:
model = read_sbml_model('models/iMM904_progesterone_fixed.xml') # Load model

In [125]:
model.add_boundary(model.metabolites.get_by_id("progesterone_c"), type="demand") # Because it is a reversible reaction, we need to set a demand for production of progesterone 

Reaction identifier,DM_progesterone_c
Name,progesterone demand
Memory address,0x13ab52da0
Stoichiometry,progesterone_c --> progesterone -->
GPR,
Lower bound,0
Upper bound,1000.0


In [41]:
#cobra.io.save_json_model(model, "progesterone_yeast_model.json") # Save model as .json file to be uploaded in Escher

In [133]:
escher.Builder('iMM904.Central carbon metabolism') # The iMM904 model

Builder()

In [134]:
escher.Builder('iMM904.Central carbon metabolism',
               reaction_data=solution.fluxes.to_dict()) # Adding our fluxes to the iMM904 model

Builder(reaction_data={'CITtcp': 0.0, '13BGH': 0.0, '13BGHe': 0.0, '13GS': 0.32667000056331613, '16GS': 0.0, '…